In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import seaborn as sns
%matplotlib inline
import math
from matplotlib.ticker import MaxNLocator
plt.style.use('fivethirtyeight')
from IPython.display import HTML
from subprocess import check_output
import matplotlib
matplotlib.rcParams['figure.figsize']=(20,10)
import warnings
warnings.simplefilter('ignore')

In [3]:
#To read data uploaded int0 collab
data=pd.read_csv("dataset_pric.csv")
Rdata=data.copy()

In [4]:
#To shoe first 5 rows of data
data.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [5]:
#To show last 5 rows of data
data.tail()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.0
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.0
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.0
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.0
13319,Super built-up Area,Ready To Move,Doddathoguru,1 BHK,NaN,550,1.0,1.0,17.0


In [6]:
#To get no of rows and columns of data
data.shape

(13320, 9)

In [7]:
#To get list of columns in data
cols=list(data.columns)
cols

['area_type',
 'availability',
 'location',
 'size',
 'society',
 'total_sqft',
 'bath',
 'balcony',
 'price']

In [8]:
#Drop the columns not required
newdata = data.drop(['area_type','society','balcony','availability'],axis='columns')
Rdata=Rdata.dropna(subset=['size','total_sqft'])

In [9]:
#Checking if there are null values
newdata.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [10]:
#shape of new data
newdata.shape

(13320, 5)

In [12]:
#Dropping rows with null values
NNdata= newdata.dropna()

In [13]:
#Checking if there are null values
NNdata.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [14]:
#shape of new data
NNdata.shape

(13246, 5)

In [15]:
#To check data types of all elements
NNdata.dtypes

location       object
size           object
total_sqft     object
bath          float64
price         float64
dtype: object

In [16]:
#Unique values in Non NUll data size column
NNdata['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [17]:
#Adding new bhk column of int data type to convert string data type of size column
NNdata['bhk'] = NNdata['size'].apply(lambda x: int(x.split(' ')[0]))
Rdata['bhk'] = Rdata['size'].apply(lambda x: int(x.split(' ')[0]))
NNdata.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [18]:
#checking Retrieval data
Rdata.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price,bhk
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07,2
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00,4
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00,3
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00,3
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00,2


In [19]:
#Unique values in Non NUll data Total_sqft column

NNdata['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [20]:
#All are string values and some even have a range which is to be corrected
#lets define a function so that it can be reused for every row
def convert_sqft_to_float(x):

  tokens = x.split('-')
  if len(tokens) == 2:
    return(float(tokens[0])+float(tokens[1]))/2
  try:
    return float(x)
  except:
    return None


In [21]:
#Applying the function to every row
NNdata['total_sqft'] = NNdata['total_sqft'].apply(convert_sqft_to_float)
Rdata['total_sqft'] = Rdata['total_sqft'].apply(convert_sqft_to_float)

In [22]:
#A function to check if there are any floats
def is_float(x):
   try:
     float(x)
   except:
     return False
   return True

In [23]:
NNdata[~NNdata['total_sqft'].apply(is_float)].head(12)

,location,size,total_sqft,bath,price,bhk


In [24]:
# Creating a new column to find the price per sq-ft
#We multiply the price here with 100000 since its in lakhs
NNdata['Price_per_sqft'] = NNdata['price']*100000/NNdata['total_sqft']
NNdata.head()

,location,size,total_sqft,bath,price,bhk,Price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [25]:
#Refining location column so that all the locations with less no of properties can be categeorized to 1 so to avoid imporper training
NNdata['location'].unique()

array(['Electronic City Phase II', 'Chikka Tirupathi', 'Uttarahalli', ...,
       '12th cross srinivas nagar banshankari 3rd stage',
       'Havanur extension', 'Abshot Layout'], dtype=object)

In [26]:
#to remove any left spaces and right spaces in locations
NNdata['location'] = NNdata['location'].apply(lambda x: x.strip())


In [27]:
#To check count of no of rows for each location
locations = NNdata.groupby('location')['location'].agg('count').sort_values(ascending=False)
locations

location
Whitefield               535
Sarjapur  Road           392
Electronic City          304
Kanakpura Road           266
Thanisandra              236
                        ... 
1 Giri Nagar               1
Kanakapura Road,           1
Kanakapura main  Road      1
Karnataka Shabarimala      1
whitefiled                 1
Name: location, Length: 1293, dtype: int64

In [28]:
#lets group all the locations with very low numbers into others categeory
NNdata['location'] = NNdata.location.apply(lambda x: 'other' if x in locations[locations<10] else x)

In [29]:
#Removing exceptional houses with abnormally high bedroom size OUTLIERS
NNdata=NNdata[~(NNdata['total_sqft']/NNdata['bhk']<300)]

In [30]:
NNdata.Price_per_sqft.describe()

count     12456.000000
mean       6308.502826
std        4168.127339
min         267.829813
25%        4210.526316
50%        5294.117647
75%        6916.666667
max      176470.588235
Name: Price_per_sqft, dtype: float64

In [31]:
#Since max price per sqft is abnormally high
#we need to remove those outliers also
def remove_pps_outliers(df):
  df_out = pd.DataFrame()
  for key, subdf in df.groupby('location'):
    m = np.mean(subdf.Price_per_sqft)
    st = np.std(subdf.Price_per_sqft)
    reduced_df = subdf[(subdf.Price_per_sqft>(m-st)) & (subdf.Price_per_sqft<=(m+st))]
    df_out = pd.concat([df_out, reduced_df],ignore_index=True)
  return df_out

NNdata= remove_pps_outliers(NNdata)

In [32]:
NNdata.head()

,location,size,total_sqft,bath,price,bhk,Price_per_sqft
0,1st Block Jayanagar,4 BHK,2850.0,4.0,428.0,4,15017.543860
1,1st Block Jayanagar,3 BHK,1630.0,3.0,194.0,3,11901.840491
2,1st Block Jayanagar,3 BHK,1875.0,2.0,235.0,3,12533.333333
3,1st Block Jayanagar,3 BHK,1200.0,2.0,130.0,3,10833.333333
4,1st Block Jayanagar,2 BHK,1235.0,2.0,148.0,2,11983.805668


In [33]:
#Drop size column and price per sft column since sizer per sft can be calculated by total price /sft
#and size since we already added bhk column
NNdata =NNdata.drop(['size','Price_per_sqft'],axis='columns')
NNdata.head()



,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [35]:
#Use one hot encoding since it will be better while taking user inputs than standard scaler
dummies = pd.get_dummies(NNdata.location)
dummies.head()

,1st Block Jayanagar,1st Block Koramangala,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,6th Phase JP Nagar,7th Phase JP Nagar,8th Phase JP Nagar,...,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur,other
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
NNdata= pd.concat([NNdata, dummies],axis="columns")
NNdata.head()

,location,total_sqft,bath,price,bhk,1st Block Jayanagar,1st Block Koramangala,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,...,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur,other
0,1st Block Jayanagar,2850.0,4.0,428.0,4,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1st Block Jayanagar,1630.0,3.0,194.0,3,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1st Block Jayanagar,1875.0,2.0,235.0,3,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1st Block Jayanagar,1200.0,2.0,130.0,3,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1st Block Jayanagar,1235.0,2.0,148.0,2,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [37]:
#since other is having NUll values lets remove it also along with location column
NNdata=NNdata.drop(['location'],axis='columns')
NNdata.head()

,total_sqft,bath,price,bhk,1st Block Jayanagar,1st Block Koramangala,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,...,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur,other
0,2850.0,4.0,428.0,4,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1630.0,3.0,194.0,3,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1875.0,2.0,235.0,3,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1200.0,2.0,130.0,3,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1235.0,2.0,148.0,2,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [38]:
NNdata.shape

(10222, 259)

In [39]:
#Seperaring independent variables into X
X =NNdata.drop(['price'], axis='columns')
X

,total_sqft,bath,bhk,1st Block Jayanagar,1st Block Koramangala,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,...,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur,other
0,2850.0,4.0,4,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1630.0,3.0,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1875.0,2.0,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1200.0,2.0,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1235.0,2.0,2,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10217,1353.0,2.0,2,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10218,812.0,1.0,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10219,1440.0,2.0,3,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10220,1075.0,2.0,2,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
#dependent variables into Y
Y =NNdata['price']
Y

In [ ]:
#splitting into training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=10)

In [ ]:
#Testing with different algorithms
All_Algos=[]


In [ ]:
#1 linear regression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
LR= LinearRegression()
LR.fit(X_train, y_train)
LRA=LR.score(X_test,y_test)
LRA=LRA*100
CvLR=cross_val_score(LinearRegression(),X,Y, cv=cv)
LRAvg=(sum(CvLR)/len(CvLR))*100
print("Linear regression accuracy is",LRA,"\n and the avg accuracy for linear regression after cross val is ",LRAvg )
All_Algos.append([LRAvg,'LinearRegression'])
print("added for comparison ")

In [ ]:
#2 Decision Tree
from sklearn.tree import DecisionTreeRegressor
DT= DecisionTreeRegressor(random_state=0)
DT.fit(X_train, y_train)
DTA=DT.score(X_test,y_test)
DTA=DTA*100
CvDT=cross_val_score(DecisionTreeRegressor(),X,Y, cv=cv)
DTAvg=(sum(CvDT)/len(CvDT))*100
print("Decision Tress Accuracy is",DTA,"\n and the avg accuracy for Decision Tree after cross val is ",DTAvg )
All_Algos.append([DTAvg,"DecisionTree"])
print("added for comparison ")

In [ ]:
#3 MLP regressor
from sklearn.neural_network import MLPRegressor
MLPR= MLPRegressor(random_state=1, max_iter=500)
MLPR.fit(X_train, y_train)
MLPRA=MLPR.score(X_test,y_test)
MLPRA=MLPRA*100
CvMLPR=cross_val_score(MLPRegressor(random_state=1, max_iter=500),X,Y, cv=cv)
MLPRAvg=(sum(CvMLPR)/len(CvMLPR))*100
print("MLPRegressor Accuracy is",MLPRA,"\n and the avg accuracy for MLPRegressor after cross val is ",MLPRAvg )
All_Algos.append([MLPRAvg,"MLPRegressor"])
print("added for comparison ")

MLPRegressor Accuracy is 67.23832066319353 
 and the avg accuracy for MLPRegressor after cross val is  69.38528238600726
added for comparison 


In [ ]:
#4 Ridge regression
from sklearn.linear_model import Ridge
RR= Ridge(alpha=1.0)
RR.fit(X_train, y_train)
RRA=RR.score(X_test,y_test)
RRA=RRA*100
CvRR=cross_val_score(Ridge(alpha=1.0),X,Y, cv=cv)
RRAvg=(sum(CvRR)/len(CvRR))*100
print("MLPRegressor Accuracy is",RRA,"\n and the avg accuracy for MLPRegressor after cross val is ",RRAvg )
All_Algos.append([RRAvg,"RRegressor"])
print("added for comparison ")

MLPRegressor Accuracy is 82.39882432762593 
 and the avg accuracy for MLPRegressor after cross val is  81.17634007086941
added for comparison 


In [ ]:
#5 Linear model Lasso Regression
from sklearn import linear_model
LMLR=linear_model.Lasso(alpha=0.1)
LMLR.fit(X_train, y_train)
LMLRA=LMLR.score(X_test,y_test)
LMLRA=LMLRA*100
CvLMLR=cross_val_score(linear_model.Lasso(alpha=0.1),X,Y, cv=cv)
LMLRAvg=(sum(CvLMLR)/len(CvLMLR))*100
print("LassoRegressor Accuracy is",LMLRA,"\n and the avg accuracy for LassoRegressor after cross val is ",LMLRAvg )
All_Algos.append([LMLRAvg,"LassoRegressor"])
print("added for comparison ")


LassoRegressor Accuracy is 79.29127004314927 
 and the avg accuracy for LassoRegressor after cross val is  76.79967851315777
added for comparison 


In [ ]:
#6 Random Forest
from sklearn.ensemble import RandomForestRegressor
RF= RandomForestRegressor(max_depth=10, random_state=0)
RF.fit(X_train, y_train)
RFA=RF.score(X_test,y_test)
RFA=RFA*100
CvRF=cross_val_score(RandomForestRegressor(max_depth=10, random_state=0),X,Y, cv=cv)
RFAvg=(sum(CvRF)/len(CvRF))*100
print("RandomForest Accuracy is",RFA,"\n and the avg accuracy for RandomForest after cross val is ",RFAvg )
All_Algos.append([RFAvg,"RandomForest"])
print("added for comparison ")


RandomForest Accuracy is 77.89716051278513 
 and the avg accuracy for RandomForest after cross val is  75.14938173317653
added for comparison 


In [ ]:
#7KNNRegressor
from sklearn.neighbors import KNeighborsRegressor
KNN= KNeighborsRegressor(n_neighbors=4)
KNN.fit(X_train, y_train)
KNNA=KNN.score(X_test,y_test)
KNNA=KNNA*100
CvKNN=cross_val_score(KNeighborsRegressor(n_neighbors=4),X,Y, cv=cv)
KNNAvg=(sum(CvKNN)/len(CvKNN))*100
print("KNNRegressor Accuracy is",KNNA,"\n and the avg accuracy for KNNRegressor after cross val is ",KNNAvg )
All_Algos.append([KNNAvg,"KNNRegressor"])
print("added for comparison ")

KNNRegressor Accuracy is 69.21511443144273 
 and the avg accuracy for KNNRegressor after cross val is  65.68129571599472
added for comparison 


In [ ]:
#To find out the best Accuracy algorithm
Best=sorted(All_Algos)[::-1][0]
All_Names=['LinearRegression',"DecisionTree","MLPRegressor","RRegressor",'LassoRegressor',"RandomForest","KNNRegressor"]
All_models=[LR,DT,MLPR,RR,LMLR,RF,KNN]
BA=All_models[All_Names.index(Best[1])]
print(Best[1],"is best algorithm with vg accuracy",Best[0])
print("The selected model is ",BA)

RRegressor is best algorithm with vg accuracy 81.17634007086941
The selected model is  Ridge()


In [ ]:
#Function to predict values
def predict_price(location, sqft,bath, bhk):

  try:
    loc_index=np.where(X.columns==location)[0][0]
    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0 :
      x[loc_index] = 1
    return BA.predict([x])[0]
  except:
    print("please check inputs")
    return 0

In [ ]:
1#To take inputs
pred=0
while(pred==0):
  try:
    l=str(input("Enter the location"))
    s=int(input("Enter required sq ft"))
    b=int(input("enter no of bathrooms"))
    bh=int(input("enter the no of bedrooms"))
    pred=predict_price(l,s,b,bh)
  except:
    print("Please check input data type")
print("The predicted price for this loaction is ",pred*100000,"₹")


**Prediction done, Now to retrieve and show data to the user based on the given data set**




In [ ]:
#To check retrieval Data
Rdata['price']=Rdata["price"]*100000
Rdata.head()

In [ ]:
#The list of properties close to given size and in specified  place, with required  bedrooms and bathrooms specifications
Fa=int(input("enter range of change in area thats ok for user"))
List_of_all_properties_with_specs=Rdata[(Rdata['location']==l)&(Rdata['bhk']==bh)&(Rdata['bath']==b)&(Rdata['total_sqft']<=(s+Fa))&(Rdata['total_sqft']>=(s-Fa))]
List_of_all_properties_with_specs

In [ ]:
#Returning all the ready to move into properties in specified area with price less than or equal to predicted price
Predicted_Price_per_sqft=(pred*100000)/s
List_of_all_properties_ready_to_move_with_predicted_ppsft=List_of_all_properties_with_specs[(List_of_all_properties_with_specs['availability']=="Ready To Move")&((List_of_all_properties_with_specs['price'])<=(List_of_all_properties_with_specs['total_sqft']*Predicted_Price_per_sqft))]
List_of_all_properties_ready_to_move_with_predicted_ppsft

In [ ]:
#To return all properties which are over priced
List_of_all_properties_over_priced=List_of_all_properties_with_specs[((List_of_all_properties_with_specs['price'])>(List_of_all_properties_with_specs['total_sqft']*Predicted_Price_per_sqft))]
List_of_all_properties_over_priced

In [ ]:
#Check dummies columns for all location names
Locs=list(dummies.columns)

In [ ]:
#user inputs the area,bedrooms,bathrooms and his budget and areas he can afford are shown
budget=int(input("Enter the budget"))
Locations_in_budget=[]
for i in Locs:
  pred1=predict_price(i,s,b,bh)
  if((pred1*100000)<=budget):
    Locations_in_budget.append(i)
    print(i)
print("No of areas in budget =",len(Locations_in_budget))

In [ ]:
#returning all the properties with specified parameters with price less than or equal to budget
List_of_all_properties_with_specs_in_budget_locations=Rdata[(Rdata['location'].isin(Locations_in_budget))&(Rdata['bhk']==bh)&(Rdata['bath']==b)&(Rdata['total_sqft']<=(s+Fa))&(Rdata['total_sqft']>=(s-Fa))&(Rdata['price']<=budget)]
List_of_all_properties_with_specs_in_budget_locations